In [14]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [15]:
!pip install kaggle

In [16]:
!pwd

/content


In [17]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [18]:
!mkdir -p ~/.kaggle
!cp '/content/drive/My Drive/kaggle.json' ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [19]:
!kaggle competitions download -c digit-recognizer

 76% 7.00M/9.16M [00:00<00:00, 52.9MB/s]
100% 9.16M/9.16M [00:00<00:00, 44.8MB/s]
  0% 0.00/6.09M [00:00<?, ?B/s]
100% 6.09M/6.09M [00:00<00:00, 99.8MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 70.8MB/s]


In [20]:
!unzip 'train.csv.zip'
!unzip 'test.csv.zip'

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [21]:
!rm -rf test.csv.zip
!rm -rf train.csv.zip


In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from subprocess import check_output
print(check_output(["ls","../content"]).decode("utf8"))

drive
sample_data
sample_submission.csv
test.csv
train.csv



In [23]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [25]:
trainy = train['label'].astype('float32')
trainx = train.drop(['label'],axis=1).astype('int32')
testx = test.astype('float32')
trainx.shape,trainy.shape,testx.shape

((42000, 784), (42000,), (28000, 784))

In [27]:
trainx = trainx.values.reshape(-1,28,28,1)
trainx = trainx / 255.0
testx = testx.values.reshape(-1,28,28,1)
testx = testx / 255.0
trainx.shape , testx.shape

AttributeError: ignored

In [28]:
trainy = tf.keras.utils.to_categorical(trainy,10)
trainy.shape

(42000, 10)

In [29]:
print(train['label'].head())

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64


In [30]:
print(trainy[0:5,:])

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [31]:
model = tf.keras.models.Sequential([
   tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape = (28,28,1)),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dropout(0.50),
    tf.keras.layers.Dense(10,activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 6, 64)         

In [32]:
class mycallback(tf.keras.callbacks.callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('accuracy')>0.999):
      print("reached 99.9%")
      self.model.stop_training = True
callbacks = mycallback()

AttributeError: ignored

In [33]:
class mycallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs={}):
    if(logs.get('accuracy')>0.999):
      print("reached 99.9%")
      self.model.stop_training = True
callbacks = mycallback()
optimizer = tf.keras.optimizers.Adam(
    learning_rate = 0.0005,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'Adam'
)
model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics = ['accuracy'])
model.fit(trainx,trainy,batch_size=50,epochs = 20,callbacks=[callbacks])

Epoch 1/20
840/840 [==============================] - 126s 149ms/step - loss: 0.7748 - accuracy: 0.7269
Epoch 2/20
840/840 [==============================] - 129s 153ms/step - loss: 0.0926 - accuracy: 0.9725
Epoch 3/20
840/840 [==============================] - 130s 155ms/step - loss: 0.0613 - accuracy: 0.9810
Epoch 4/20
840/840 [==============================] - 130s 154ms/step - loss: 0.0511 - accuracy: 0.9856
Epoch 5/20
840/840 [==============================] - 129s 154ms/step - loss: 0.0426 - accuracy: 0.9877
Epoch 6/20
840/840 [==============================] - 129s 154ms/step - loss: 0.0359 - accuracy: 0.9895
Epoch 7/20
840/840 [==============================] - 130s 155ms/step - loss: 0.0282 - accuracy: 0.9917
Epoch 8/20
840/840 [==============================] - 130s 155ms/step - loss: 0.0263 - accuracy: 0.9922
Epoch 9/20
840/840 [==============================] - 124s 148ms/step - loss: 0.0209 - accuracy: 0.9938
Epoch 10/20
840/840 [==============================] - 123s 146m

In [ ]:
results = model.predict(testx)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")

KeyboardInterrupt: ignored

In [ ]:
submitt = pd.concat([pd.Series(range(1,28001),name="ImageId"),results],axis=1) 
submitt.to_csv("newfile.csv",index=False)

In [ ]:
!kaggle competitions submit -c digit-recognizer -f newfile.csv -m "message"

100% 208k/208k [00:01<00:00, 140kB/s]
Successfully submitted to Digit Recognizer

# New Section